In [ ]:
TELEGRAM_BOT_TOKEN = ''
TELEGRAM_CHAT_ID = ''

In [ ]:
import gzip
import pandas as pd
import os
import tarfile 
import zipfile
import tensorflow as tf
import time

In [ ]:
base_dir = '.'
dataset_folder = os.path.join(base_dir,'license-blobs')

In [ ]:
import gzip
import shutil

def gunzip_shutil(source_filepath, dest_filepath, block_size=65536):
    with gzip.open(source_filepath, 'rb') as s_file, \
            open(dest_filepath, 'wb') as d_file:
        shutil.copyfileobj(s_file, d_file, block_size)

def is_gz_file(filepath):
    with open(filepath, 'rb') as test_f:
        return test_f.read(2) == b'\x1f\x8b'

# try if everything is ok
filename1 = os.path.join(base_dir, "license-blobs/00/0a/000a799c27dfbad9a943dbf585d00f0cbb81a32c.tmp.tmp")
if os.path.exists(filename1) and is_gz_file(filename1):
    gunzip_shutil(filename1, filename1+".tmp")

In [ ]:
!pip install charset_normalizer
from charset_normalizer import detect

def get_encoding_filename(filename):
    try:
        return detect(open(filename, 'rb').read()).get('encoding')
    except:
        return None

In [ ]:
num_file = 0
from collections import Counter
cnt = Counter()
def extract(corpus_path):
    global num_file
    corpus_path, dirs, files = next(os.walk(corpus_path))

    corpus_path, dirs, files1 = next(os.walk(corpus_path))
    for D1 in dirs:
       
        dir_path = os.sep.join([corpus_path,D1])
        print(f"{time.strftime('%H:%M:%S')} Current path: {dir_path}")
        dir_path, dirs2, files2 = next(os.walk(dir_path))
        for D2 in dirs2:
          
          dir_path2 = os.sep.join([dir_path, D2])
          print(f"{time.strftime('%H:%M:%S')} Current folder: {dir_path2}")
          dir_path2, dirs3, files3 = next(os.walk(dir_path2))

          for f in files3:
            try:
              data_path = os.sep.join([dir_path2, f])
              
              # extract file
              if is_gz_file(data_path):
                    gunzip_shutil(data_path, data_path+".tmp")
                    # remove old file
                    os.remove(data_path)
                    data_path += ".tmp"
              num_file += 1
              cnt[get_encoding_filename(data_path)] += 1
            except Exception as e:
              print(e)
        print(f"{time.strftime('%H:%M:%S')} Numero di file letti: {num_file}")
        print(f"{time.strftime('%H:%M:%S')} {cnt.most_common(10)}")
extract(dataset_folder)
print(f"{time.strftime('%H:%M:%S')} {cnt.most_common(10)}")

In [ ]:
# Download dataset and extract
if not os.path.exists(os.path.abspath('.') + dataset_folder):
  dataset_tar = tf.keras.utils.get_file('license-blobs.tar',
                                      cache_subdir=os.path.abspath('.')+dataset_folder,
                                      origin = 'https://casacalarota.onthewifi.com/wp-content/uploads/2020/10/RDC/license-blobs.tar',
                                      extract = True)
  PATH = os.path.dirname(dataset_tar) + dataset_folder
  os.remove(dataset_tar)
  extract(dataset_folder)
else:
  PATH = os.path.abspath('.') + dataset_folder

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
def save_df_to_csv(df, filename):
  df.to_csv(os.path.join(base_dir, filename))

In [ ]:
def reader(corpus_path):
    corpus_path, dirs, files = next(os.walk(corpus_path))
    if "license-blobs" in dirs:
        corpus_path = os.sep.join([corpus_path,"license-blobs"])
        #otherwise assuming we're already in rcv1. 
        #just want to make the behavior robust here.
        # you can pass either a Reuters corpus dir with RCV1 or RCV1 itself
        # Don't care which.

    corpus_path, dirs, files1 = next(os.walk(corpus_path))
    for D1 in dirs:
        result = []
        dir_path = os.sep.join([corpus_path,D1])
        dir_path, dirs2, files2 = next(os.walk(dir_path))
        for D2 in dirs2:
          dir_path2 = os.sep.join([dir_path, D2])
          dir_path2, dirs3, files3 = next(os.walk(dir_path2))

          for f in files3:
            try:
              data_path = os.sep.join([dir_path2, f])
              with open(data_path, 'r', encoding="utf8") as f3:
                  raw_data = f3.read()
              

              def parse_text(raw_data): 
                  # HERE PUT STOPWORDS parsing
                  return ' '.join([word for word in raw_data.split() if str(word).lower() not in (stop)])
            
              
              #assemble output
              output = {
                        "text": parse_text(raw_data),
                        "path": data_path
                        }

              result.append(output)
            except Exception as e:
              print(e)
        print(f"Saving {D1} results..")
        df = pd.DataFrame(result)
        df.dropna(inplace=True,subset=['text'])
        print(df.head())
        if not os.path.exists(os.path.join(base_dir, 'dataset_parsed')):
            os.makedirs(os.path.join(base_dir,'dataset_parsed'))
        save_df_to_csv(df, f'dataset_parsed/{D1}_total_no_stopwords.csv')
    return result

In [ ]:
reader(base_dir)

In [ ]:
def send_telegram_message(text):
    if TELEGRAM_BOT_TOKEN and TELEGRAM_CHAT_ID:
        !pip install -q python-telegram-bot
        from telegram import Bot
        bot = Bot(token=TELEGRAM_BOT_TOKEN)
        bot.send_message(text=text, chat_id=TELEGRAM_CHAT_ID)
send_telegram_message("Finito di estrarre dataset completo")

In [ ]:
from collections import Counter
file_num = 0
words_set = set()
cnt = Counter()
def calculate_file_num(corpus_path):
    global file_num
    corpus_path, dirs, files = next(os.walk(corpus_path))
    if "dataset_parsed" in dirs:
        corpus_path = os.sep.join([corpus_path,"dataset_parsed"])
    
    for f in files:
        print(f)
        try:
            data_path = os.sep.join([corpus_path, f])
            document = pd.read_csv(data_path)
            document.dropna(subset=['text'])
            file_num += len(document['text'])
            for index, row in document.iterrows():
                # print(row['text'])
                if isinstance(row['text'], str):
                    for word in row['text'].split(' '):
                        words_set.add(word)
                        cnt[word] += 1
            print("numero testi: {}".format(len(document['text'])))
            print("lunghezza set parole: {}".format(len(words_set)))
        except Exception as e:
            print(e)

In [ ]:
calculate_file_num("dataset_parsed")

In [ ]:
print(f"numero testi: {file_num}")
print(f"lunghezza set parole: {len(words_set)}")
print(cnt.most_common(10))

In [ ]:
print(cnt.most_common(50))

In [ ]:
send_telegram_message(f"Finito di salvare stat dataset:\nnumero testi: {file_num}\nlunghezza set parole: {len(words_set)}\n{cnt.most_common(10)}")